In [115]:
# Import libraries
import IPython
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,StratifiedKFold
# Set up any configuration, options, or magic commands
%matplotlib inline

In [116]:
import warnings
warnings.filterwarnings('ignore')

In [117]:
# import os
# path = os.getcwd()
# path

In [118]:
df_mc = pd.read_csv("Dataset_SMOTE_BC.csv")
df_mc = df_mc.drop('binary_target', axis=1)
df_mc

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,target
0,120.000000,0.000,0.000000,0.000000,0.000000,0.0,0.0,73.000000,0.500000,43.000000,...,62.000000,126.000000,2.000000,0.000000,120.000000,137.000000,121.000000,73.000000,1.000000,2.0
1,132.000000,0.005,0.000000,0.006000,0.003000,0.0,0.0,17.000000,2.100000,0.000000,...,68.000000,198.000000,6.000000,0.569969,141.000000,136.000000,140.000000,12.000000,0.000000,1.0
2,133.000000,0.003,0.000000,0.008000,0.003000,0.0,0.0,16.000000,2.100000,0.000000,...,68.000000,198.000000,5.000000,0.569969,141.000000,135.000000,138.000000,13.000000,0.000000,1.0
3,134.000000,0.003,0.000000,0.008000,0.003000,0.0,0.0,16.000000,2.400000,0.000000,...,53.000000,170.000000,11.000000,0.000000,137.000000,134.000000,137.000000,13.000000,1.000000,1.0
4,132.000000,0.005,0.000000,0.008000,0.000000,0.0,0.0,16.000000,2.400000,0.000000,...,53.000000,170.000000,9.000000,0.000000,137.000000,136.000000,138.000000,11.000000,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4960,144.969611,0.000,0.002657,0.000000,0.000000,0.0,0.0,80.198587,0.211449,67.457951,...,136.771025,149.771025,1.000000,0.000000,145.313074,143.656537,145.542049,1.000000,0.000000,3.0
4961,129.535356,0.000,0.010000,0.003384,0.003384,0.0,0.0,34.383839,2.515152,0.000000,...,54.000000,199.000000,10.232322,0.569969,89.061035,94.929289,86.500000,76.000000,-1.000000,3.0
4962,130.212257,0.000,0.003650,0.003825,0.000912,0.0,0.0,73.962797,1.366849,3.631290,...,72.493440,135.018602,1.368710,0.000000,126.106129,122.456237,126.193656,6.649892,1.000000,3.0
4963,125.321477,0.000,0.001107,0.002369,0.002952,0.0,0.0,77.726099,0.942864,3.154735,...,67.904848,135.904848,2.583371,0.000000,121.630947,119.785682,121.892841,7.428636,0.630947,3.0


In [119]:
X = df_mc.iloc[:,0:21]
y = df_mc.iloc[:,-1]

In [120]:
y

0       2.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
4960    3.0
4961    3.0
4962    3.0
4963    3.0
4964    3.0
Name: target, Length: 4965, dtype: float64

In [121]:
# Converting float_values to integers using astype()
y = y.astype(int)
y = y.map({1: 0, 2: 1, 3: 2})
y

0       1
1       0
2       0
3       0
4       0
       ..
4960    2
4961    2
4962    2
4963    2
4964    2
Name: target, Length: 4965, dtype: int64

In [95]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [96]:
xgboost = xgb.XGBClassifier()

# hyperparameter tuning
param_grid = {
    "learning_rate": [0.0001,0.001, 0.01, 0.1, 1] ,
    "max_depth": range(3,21,3),
    "gamma": [i/10.0 for i in range(0,5)],
    "colsample_bytree": [i/10.0 for i in range(3,10)],
    "reg_alpha": [1e-5, 1e-2, 0.1, 1, 10, 100],
    "reg_lambda": [1e-5, 1e-2, 0.1, 1, 10, 100]}

random_search=RandomizedSearchCV(xgboost ,param_distributions=param_grid,n_iter=5,scoring='accuracy',n_jobs=-1,cv=5,verbose=3)
random_search.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/Users/muntaqimahmedraju/miniforge3/envs/pythonhub/lib/python3.9/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/muntaqimahmedraju/miniforge3/envs/pythonhub/lib/python3.9/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Users/muntaqimahmedraju/miniforge3/envs/pythonhub/lib/python3.9/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Users/muntaqimahmedraju/miniforge3/envs/pythonhub/lib/python3.9/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a f

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5, 0.6,
                                                             0.7, 0.8, 0.9],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.0001, 0.001, 0.01,
                                                          0.1, 1],
                                        'max_depth': range(3, 21, 3),
                                        'reg_alpha': [1e-05, 0.01, 0.1, 1, 10,
                                                      100],
                                        'reg_lambda': [1e-05, 0.01, 0.1, 1, 10,
                                                       100]},
                   scoring='accuracy', verbose=3)

In [97]:
random_search.best_params_

{'reg_lambda': 0.1,
 'reg_alpha': 1,
 'max_depth': 18,
 'learning_rate': 0.1,
 'gamma': 0.2,
 'colsample_bytree': 0.7}

In [98]:
xgboost = xgb.XGBClassifier(
reg_lambda = 0.1,
reg_alpha = 1e-05,
max_depth = 12,
learning_rate = 0.0001,
gamma = 0.0,
colsample_bytree = 0.6
)
xgboost.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0001, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [99]:
y_pred_test = xgboost.predict(X_test)
print(accuracy_score(y_test,y_pred_test)*100)

97.71812080536913


In [100]:
lgboost = lgb.LGBMClassifier()

param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],  # Learning rate (shrinkage)
    'n_estimators': [50, 100, 200],  # Number of boosting iterations
    'num_leaves': [15, 31, 63],  # Maximum number of leaves in a tree
    'max_depth': [5, 10, 15],  # Maximum tree depth for depth-wise tree growth
    'subsample': [0.6, 0.8, 1.0],  # Subsample ratio of the training instance
    'colsample_bytree': [0.6, 0.8, 1.0],  # Subsample ratio of columns when constructing each tree
    'min_child_samples': [20, 50, 100],  # Minimum number of data needed in a leaf
    'reg_alpha': [0.0, 0.1, 0.5],  # L1 regularization term on weights
    'reg_lambda': [0.0, 0.1, 0.5],  # L2 regularization term on weights
}
random_search=RandomizedSearchCV(lgboost ,param_distributions=param_grid,n_iter=5,scoring='accuracy',n_jobs=-1,cv=5,verbose=3)
random_search.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000695 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4645
[LightGBM] [Info] Number of data points in the train set: 3475, number of used features: 19
[LightGBM] [Info] Start training from score -1.099764
[LightGBM] [Info] Start training from score -1.085178
[LightGBM] [Info] Start training from score -1.111064
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

RandomizedSearchCV(cv=5, estimator=LGBMClassifier(), n_iter=5, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'learning_rate': [0.01, 0.05, 0.1],
                                        'max_depth': [5, 10, 15],
                                        'min_child_samples': [20, 50, 100],
                                        'n_estimators': [50, 100, 200],
                                        'num_leaves': [15, 31, 63],
                                        'reg_alpha': [0.0, 0.1, 0.5],
                                        'reg_lambda': [0.0, 0.1, 0.5],
                                        'subsample': [0.6, 0.8, 1.0]},
                   scoring='accuracy', verbose=3)

In [101]:
random_search.best_params_

{'subsample': 0.8,
 'reg_lambda': 0.5,
 'reg_alpha': 0.1,
 'num_leaves': 31,
 'n_estimators': 200,
 'min_child_samples': 50,
 'max_depth': 10,
 'learning_rate': 0.1,
 'colsample_bytree': 1.0}

In [102]:
lgboost = lgb.LGBMClassifier(
subsample = 1.0,
reg_lambda= 0.0,
reg_alpha =0.5,
num_leaves = 15,
n_estimators = 200,
min_child_samples = 20,
max_depth = 10,
learning_rate = 0.1,
colsample_bytree = 0.6
)
lgboost.fit(X_train,y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000675 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4645
[LightGBM] [Info] Number of data points in the train set: 3475, number of used features: 19
[LightGBM] [Info] Start training from score -1.099764
[LightGBM] [Info] Start training from score -1.085178
[LightGBM] [Info] Start training from score -1.111064
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

LGBMClassifier(colsample_bytree=0.6, max_depth=10, n_estimators=200,
               num_leaves=15, reg_alpha=0.5)

In [103]:
y_pred_test = lgboost.predict(X_test)
print(accuracy_score(y_test,y_pred_test)*100)

97.8523489932886


### Binary Class

In [122]:
df_bc = pd.read_csv("Dataset_SMOTE_BC.csv")
df_bc = df_bc.drop('target', axis=1)
df_bc

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,binary_target
0,120.000000,0.000,0.000000,0.000000,0.000000,0.0,0.0,73.000000,0.500000,43.000000,...,62.000000,126.000000,2.000000,0.000000,120.000000,137.000000,121.000000,73.000000,1.000000,0
1,132.000000,0.005,0.000000,0.006000,0.003000,0.0,0.0,17.000000,2.100000,0.000000,...,68.000000,198.000000,6.000000,0.569969,141.000000,136.000000,140.000000,12.000000,0.000000,1
2,133.000000,0.003,0.000000,0.008000,0.003000,0.0,0.0,16.000000,2.100000,0.000000,...,68.000000,198.000000,5.000000,0.569969,141.000000,135.000000,138.000000,13.000000,0.000000,1
3,134.000000,0.003,0.000000,0.008000,0.003000,0.0,0.0,16.000000,2.400000,0.000000,...,53.000000,170.000000,11.000000,0.000000,137.000000,134.000000,137.000000,13.000000,1.000000,1
4,132.000000,0.005,0.000000,0.008000,0.000000,0.0,0.0,16.000000,2.400000,0.000000,...,53.000000,170.000000,9.000000,0.000000,137.000000,136.000000,138.000000,11.000000,1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4960,144.969611,0.000,0.002657,0.000000,0.000000,0.0,0.0,80.198587,0.211449,67.457951,...,136.771025,149.771025,1.000000,0.000000,145.313074,143.656537,145.542049,1.000000,0.000000,0
4961,129.535356,0.000,0.010000,0.003384,0.003384,0.0,0.0,34.383839,2.515152,0.000000,...,54.000000,199.000000,10.232322,0.569969,89.061035,94.929289,86.500000,76.000000,-1.000000,0
4962,130.212257,0.000,0.003650,0.003825,0.000912,0.0,0.0,73.962797,1.366849,3.631290,...,72.493440,135.018602,1.368710,0.000000,126.106129,122.456237,126.193656,6.649892,1.000000,0
4963,125.321477,0.000,0.001107,0.002369,0.002952,0.0,0.0,77.726099,0.942864,3.154735,...,67.904848,135.904848,2.583371,0.000000,121.630947,119.785682,121.892841,7.428636,0.630947,0


In [105]:
X = df_bc.iloc[:,0:21]
y = df_bc.iloc[:,-1]

In [106]:
y

0       0
1       1
2       1
3       1
4       1
       ..
4960    0
4961    0
4962    0
4963    0
4964    0
Name: binary_target, Length: 4965, dtype: int64

In [107]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [108]:
xgboost = xgb.XGBClassifier()

# hyperparameter tuning
param_grid = {
    "learning_rate": [0.0001,0.001, 0.01, 0.1, 1] ,
    "max_depth": range(3,21,3),
    "gamma": [i/10.0 for i in range(0,5)],
    "colsample_bytree": [i/10.0 for i in range(3,10)],
    "reg_alpha": [1e-5, 1e-2, 0.1, 1, 10, 100],
    "reg_lambda": [1e-5, 1e-2, 0.1, 1, 10, 100]}

random_search=RandomizedSearchCV(xgboost ,param_distributions=param_grid,n_iter=5,scoring='accuracy',n_jobs=-1,cv=5,verbose=3)
random_search.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/Users/muntaqimahmedraju/miniforge3/envs/pythonhub/lib/python3.9/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/muntaqimahmedraju/miniforge3/envs/pythonhub/lib/python3.9/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Users/muntaqimahmedraju/miniforge3/envs/pythonhub/lib/python3.9/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Users/muntaqimahmedraju/miniforge3/envs/pythonhub/lib/python3.9/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a f

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5, 0.6,
                                                             0.7, 0.8, 0.9],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.0001, 0.001, 0.01,
                                                          0.1, 1],
                                        'max_depth': range(3, 21, 3),
                                        'reg_alpha': [1e-05, 0.01, 0.1, 1, 10,
                                                      100],
                                        'reg_lambda': [1e-05, 0.01, 0.1, 1, 10,
                                                       100]},
                   scoring='accuracy', verbose=3)

In [109]:
random_search.best_params_

{'reg_lambda': 1,
 'reg_alpha': 1e-05,
 'max_depth': 9,
 'learning_rate': 0.1,
 'gamma': 0.2,
 'colsample_bytree': 0.3}

In [110]:
xgboost = xgb.XGBClassifier(
reg_lambda = 1e-05,
reg_alpha = 0.1,
max_depth = 12,
learning_rate = 0.1,
gamma = 0.0,
colsample_bytree = 0.8
)
xgboost.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [111]:
y_pred_test = xgboost.predict(X_test)
print(accuracy_score(y_test,y_pred_test)*100)

98.79194630872483


In [57]:
lgboost = lgb.LGBMClassifier()

param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],  # Learning rate (shrinkage)
    'n_estimators': [50, 100, 200],  # Number of boosting iterations
    'num_leaves': [15, 31, 63],  # Maximum number of leaves in a tree
    'max_depth': [5, 10, 15],  # Maximum tree depth for depth-wise tree growth
    'subsample': [0.6, 0.8, 1.0],  # Subsample ratio of the training instance
    'colsample_bytree': [0.6, 0.8, 1.0],  # Subsample ratio of columns when constructing each tree
    'min_child_samples': [20, 50, 100],  # Minimum number of data needed in a leaf
    'reg_alpha': [0.0, 0.1, 0.5],  # L1 regularization term on weights
    'reg_lambda': [0.0, 0.1, 0.5],  # L2 regularization term on weights
}
random_search=RandomizedSearchCV(lgboost ,param_distributions=param_grid,n_iter=5,scoring='accuracy',n_jobs=-1,cv=5,verbose=3)
random_search.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

RandomizedSearchCV(cv=5, estimator=LGBMClassifier(), n_iter=5, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'learning_rate': [0.01, 0.05, 0.1],
                                        'max_depth': [5, 10, 15],
                                        'min_child_samples': [20, 50, 100],
                                        'n_estimators': [50, 100, 200],
                                        'num_leaves': [15, 31, 63],
                                        'reg_alpha': [0.0, 0.1, 0.5],
                                        'reg_lambda': [0.0, 0.1, 0.5],
                                        'subsample': [0.6, 0.8, 1.0]},
                   scoring='accuracy', verbose=3)

In [112]:
random_search.best_params_

{'reg_lambda': 1,
 'reg_alpha': 1e-05,
 'max_depth': 9,
 'learning_rate': 0.1,
 'gamma': 0.2,
 'colsample_bytree': 0.3}

In [113]:
lgboost = lgb.LGBMClassifier(
subsample = 1.0,
reg_lambda= 0.0,
reg_alpha =0.5,
num_leaves = 15,
n_estimators = 200,
min_child_samples = 20,
max_depth = 10,
learning_rate = 0.1,
colsample_bytree = 0.6
)
lgboost.fit(X_train,y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1157, number of negative: 2318
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000993 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4645
[LightGBM] [Info] Number of data points in the train set: 3475, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.332950 -> initscore=-0.694874
[LightGBM] [Info] Start training from score -0.694874


LGBMClassifier(colsample_bytree=0.6, max_depth=10, n_estimators=200,
               num_leaves=15, reg_alpha=0.5)

In [114]:
y_pred_test = lgboost.predict(X_test)
print(accuracy_score(y_test,y_pred_test)*100)

98.45637583892616
[CV 3/5] END colsample_bytree=0.5, gamma=0.2, learning_rate=0.1, max_depth=15, reg_alpha=100, reg_lambda=1e-05;, score=0.869 total time=   0.5s
[CV 3/5] END colsample_bytree=0.7, gamma=0.2, learning_rate=0.1, max_depth=18, reg_alpha=1, reg_lambda=0.1;, score=0.971 total time=   1.4s
[CV 3/5] END colsample_bytree=0.5, gamma=0.2, learning_rate=0.1, max_depth=15, reg_alpha=0.1, reg_lambda=0.01;, score=0.974 total time=   1.0s
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003324 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4624
[LightGBM] [Info] N